In [5]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import gzip
import json
import asyncio
from tqdm.notebook import tqdm

from apis import APIs

api = APIs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
with gzip.open("top10kdetails.json.gz", "rt") as f:
    data = pd.DataFrame.from_dict({ int(appid): fields for appid, fields in json.load(f).items() }, orient="index")

In [17]:
print(list(data[570].keys()))
all(fields["type"] == "game" for fields in data.values())

['type', 'name', 'steam_appid', 'required_age', 'is_free', 'dlc', 'detailed_description', 'about_the_game', 'short_description', 'supported_languages', 'reviews', 'header_image', 'capsule_image', 'capsule_imagev5', 'website', 'pc_requirements', 'mac_requirements', 'linux_requirements', 'developers', 'publishers', 'packages', 'package_groups', 'platforms', 'metacritic', 'categories', 'genres', 'screenshots', 'movies', 'recommendations', 'release_date', 'support_info', 'background', 'background_raw', 'content_descriptors', 'ratings']


True

In [36]:
processed_data = data.drop(columns=["type"])
processed_data["required_age"] = processed_data["required_age"].map(lambda x: 17 if x == "17+" else int(x))
processed_data

,name,steam_appid,required_age,is_free,dlc,detailed_description,about_the_game,short_description,supported_languages,reviews,...,background_raw,content_descriptors,ratings,achievements,legal_notice,controller_support,price_overview,drm_notice,ext_user_account_notice,demos
570,Dota 2,570,0,True,"[1241930, 652720]",<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...","Bulgarian, Czech, Danish, Dutch, English<stron...",“A modern multiplayer masterpiece.”<br>9.5/10 ...,...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}","{'kgrb': {'rating': '12', 'descriptors': '12 (...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
730,Counter-Strike 2,730,0,True,[2678630],"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","Czech, Danish, Dutch, English<strong>*</strong...",NaN,...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...","{'usk': {'rating': '16', 'descriptors': 'Gewal...","{'total': 1, 'highlighted': [{'name': 'A New B...",NaN,NaN,NaN,NaN,NaN,NaN
578080,PUBG: BATTLEGROUNDS,578080,0,True,NaN,"<strong>LAND, LOOT, SURVIVE!</strong><br>Play ...","<strong>LAND, LOOT, SURVIVE!</strong><br>Play ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,"English, Korean, Simplified Chinese, French, G...",NaN,...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': None}","{'kgrb': {'rating': '18', 'descriptors': '폭력성'...","{'total': 37, 'highlighted': [{'name': 'Last S...","© 2017 KRAFTON, Inc.<br />\r\nPUBG: BATTLEGROU...",NaN,NaN,NaN,NaN,NaN
1623730,Palworld,1623730,0,False,[2771110],Q. What kind of game is this?<br><br>A. In thi...,Q. What kind of game is this?<br><br>A. In thi...,"Fight, farm, build and work alongside mysterio...","English, Simplified Chinese, Traditional Chine...",NaN,...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}","{'dejus': {'rating': 'l'}, 'steam_germany': {'...","{'total': 12, 'highlighted': [{'name': 'Hillsi...",NaN,full,"{'currency': 'USD', 'initial': 2999, 'final': ...",NaN,NaN,NaN
1063730,New World,1063730,0,False,[2487480],<h1>New World - Azoth Edition</h1><p><img src=...,"Explore a thrilling, open-world MMO filled wit...","Explore a thrilling, open-world MMO filled wit...","English<strong>*</strong>, French<strong>*</st...",NaN,...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'This Game may contai...","{'esrb': {'rating': 't', 'display_online_notic...","{'total': 133, 'highlighted': [{'name': 'Repai...","© 2021, Amazon.com, Inc. or its affiliates. Al...",NaN,"{'currency': 'USD', 'initial': 3999, 'final': ...",Easy Anti-Cheat,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113940,Electronic Super Joy 2,1113940,0,True,"[1121810, 1240750]",Electronic Super Joy 2 is a brutally hard plat...,Electronic Super Joy 2 is a brutally hard plat...,Electronic Super Joy 2 is a brutally hard plat...,English,NaN,...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [5], 'notes': 'Checkpoints play graphi...",None,"{'total': 11, 'highlighted': [{'name': 'Alles ...",Copyright 2019 Two And Thirty Software Ltd,full,NaN,NaN,NaN,NaN
1095040,Unexplored 2: The Wayfarer's Legacy,1095040,0,False,"[1982030, 2000810]","<h1>Join Us on Discord</h1><p><a href=""https:/...",RPG? Adventure? Strategy? Unexplored is a genr...,"Exploration adventure set in dynamic, procedur...",English,“A great depth and intelligence of design... U...,...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",{'dejus': {'rating': 'l'}},"{'total': 25, 'highlighted': [{'name': 'Wayfar...",Unexplored 2: The Wayfarer's Legacy 2023 ©️ Lu...,NaN,"{'currency': 'USD', 'initial': 1999, 'final': ...",NaN,NaN,"[{'appid': 1769180, 'description': ''}]"

In [40]:
processed_data["categories"].iat[0]

[{'id': 1, 'description': 'Multi-player'},
 {'id': 9, 'description': 'Co-op'},
 {'id': 29, 'description': 'Steam Trading Cards'},
 {'id': 30, 'description': 'Steam Workshop'},
 {'id': 40, 'description': 'SteamVR Collectibles'},
 {'id': 35, 'description': 'In-App Purchases'},
 {'id': 8, 'description': 'Valve Anti-Cheat enabled'}]

In [41]:
processed_data.memory_usage(deep=True)

Index                         79696
name                         784575
steam_appid                   79696
required_age                  79696
is_free                        9962
dlc                          726176
detailed_description       38783736
about_the_game             34356404
short_description           3126608
supported_languages         1934165
reviews                     5228249
header_image                1327951
capsule_image               1407463
capsule_imagev5             1407463
website                      703224
pc_requirements             1911168
mac_requirements            1344512
linux_requirements          1216256
developers                   955552
publishers                   956352
packages                     831200
package_groups               891104
platforms                   1912704
metacritic                   759584
categories                  1289984
genres                       993120
screenshots                 1766400
movies                      

In [2]:
df_tags = pd.read_csv("allsteamspygames.csv", index_col=0)
df_tags

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
570,570,Dota 2,Valve,Valve,NaN,1819740,401694,0,"(200000000, 500000000)",36605,1146,1141,445,0.0,0.0,0.0,865362
730,730,Counter-Strike: Global Offensive,Valve,Valve,NaN,6992314,1019613,0,"(100000000, 200000000)",29437,644,7126,309,0.0,0.0,0.0,1203272
578080,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",NaN,1367735,977526,0,"(50000000, 100000000)",25457,832,7836,633,0.0,0.0,0.0,464497
1623730,1623730,Palworld,Pocketpair,Pocketpair,NaN,276200,18105,0,"(50000000, 100000000)",2891,475,2068,55,2999.0,2999.0,0.0,58499
1063730,1063730,New World,Amazon Games,Amazon Games,NaN,192837,81265,0,"(50000000, 100000000)",10117,216,3527,216,3999.0,3999.0,0.0,10607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2744040,2744040,没有被美女包围的三角恋,硬木工作室,硬木工作室,NaN,2,7,0,"(0, 20000)",0,0,0,0,199.0,199.0,0.0,0
1963990,1963990,Deliver Us From Evil (DUFE) - Masquerade,Galen Games,Galen Games,NaN,4,2,0,"(0, 20000)",0,0,0,0,499.0,499.0,0.0,0
2763420,2763420,Pangolin Cassowary,Ocotillo Island,Ocotillo Island,NaN,14,0,0,"(0, 20000)",0,0,0,0,999.0,999.0,0.0,1
2628400,2628400,Cozy Island Idle,"Overaction Game Studio, ReiVax",Overaction Game Studio,NaN,26,6,0,"(0, 20000)",0,0,0,0,299.0,299.0,0.0,6


In [20]:
details = {}
async def get_appdetails():
    global details
    async for appid, appdetails in api.steamspy_async(df_tags["appid"].tolist(), progressbar=tqdm):
        details[appid] = appdetails
await get_appdetails()

  0%|          | 0/68459 [00:00<?, ?it/s]

In [24]:
with gzip.open("allgamesdetails.json.gz", "wt") as f:
    json.dump(details, f)

In [25]:
details[570]["tags"]

{'Free to Play': 59515,
 'MOBA': 19946,
 'Multiplayer': 15139,
 'Strategy': 14080,
 'e-sports': 11608,
 'Team-Based': 10811,
 'Competitive': 8157,
 'Action': 7832,
 'Online Co-Op': 7350,
 'PvP': 5930,
 'Difficult': 5222,
 'Co-op': 4219,
 'RTS': 4030,
 'RPG': 3707,
 'Tower Defense': 3699,
 'Fantasy': 3681,
 'Character Customization': 2862,
 'Replay Value': 2708,
 'Action RPG': 2406,
 'Simulation': 1929}